In [29]:
import pandas as pd
import numpy as np

# Create the DataFrame directly with transformed data
df = pd.DataFrame({
    "x": [0, 2, 4, 6, 8, 10, -2, -4, -6, -8, -10],
    "sigma_x": [0.477, 0.472, 0.467, 0.462, 0.458, 0.453, 0.482, 0.487, 0.492, 0.497, 0.503],
    "y_min": [-5, -6, -7, -7.75, -8.5, -9, -3.5, -2.5, -2, -1, -0.5],
    "y_max": [-5.5, -6.5, -7.5, -8.25, -9, -9.5, -4, -3, -2.5, -1.5, -1],
    "sigma_y": [1.153, 1.154, 1.155, 1.156, 1.158, 1.159, 1.152, 1.151, 1.151, 1.15, 1.15],
    "best_x": [0.2, 1.97, 4.00, 6.3, 8.7, 10.9, -2.3, -4.3, -6.3, -8.3, -10.0],
    "best_y": [-5.73, -6.7, -7.5, -8.1, -8.6, -9.5, -3.9, -2.8, -2.1, -0.8, 0.2],
    "Time": ["18:06:46", "18:21:01", "18:29:46", "18:38:56", "18:44:49", "18:51:01", "19:01:20", "19:06:29", "19:11:58", "19:15:15", "19:18:12"]
})


In [30]:
aper = pd.DataFrame({
    "APER_TOL_1": [0.00084],
    "APER_TOL_2": [0.00036],
    "APER_TOL_3": [0.0],
    "APER_1": [0.022],
    "APER_2": [0.01715],
    "APER_3": [0.022],
    "APER_4": [0.022],
    "MECH_SEP": [-0.194],
    "S": [5693.193897]
})

def plot_2d_aperture(aper_1, aper_2, aper_3, aper_4, name):

    t = np.linspace(0, 2*np.pi, 10000)
    
    x_rect = [-aper_1, aper_1, aper_1, -aper_1, -aper_1]
    y_rect = [-aper_2, -aper_2, aper_2, aper_2, -aper_2]
    x_elipse = aper_3*np.cos(t)
    y_elipse = aper_4*np.sin(t)

    try:
        indices = np.where(abs(y_elipse)>aper_2)
        x_new = np.delete(x_elipse, indices)
        y_new = np.delete(y_elipse, indices)

        indices = np.where(abs(x_elipse)>aper_1)
        x_new = np.delete(x_new, indices)
        y_new = np.delete(y_new, indices)

        # To close the loop
        x_new = np.append(x_new, x_new[0])
        y_new = np.append(y_new, y_new[0])

        aperture_trace = go.Scatter(x=x_new, 
                                        y=y_new, 
                                        mode='lines',
                                        name=name,
                                        line=dict(color='grey', width=2))

    except: 
        print('oops')

        aper_x = aper_1
        aper_y = aper_2
            
        # Define the rectangle corners as a scatter trace
        aperture_trace = go.Scatter(
                x=[-aper_x, aper_x, aper_x, -aper_x, -aper_x],  # Close the rectangle
                y=[-aper_y, -aper_y, aper_y, aper_y, -aper_y],  # Close the rectangle
                mode='lines',
                name = name,
                line=dict(color='grey', width=2),
                fill='toself',  # To close the shape
                fillcolor='rgba(0, 0, 0, 0)'  # No fill
        )

    return aperture_trace

In [52]:
import plotly.graph_objects as go
# Compute y (average of y_min and y_max)
df["y"] = (df["y_min"] + df["y_max"]) / 2

# Create the plot
fig = go.Figure()

# Add rectangles for each data point
for i in range(len(df)):
    fig.add_shape(
        type="rect",
        x0=df["x"][i] - 4*df["sigma_x"][i], x1=df["x"][i] + 4*df["sigma_x"][i],
        y0=df["y_max"][i] - 4*df["sigma_y"][i], y1=df["y_max"][i] + 4*df["sigma_y"][i],
        line=dict(color="blue"),
        fillcolor="rgba(0, 0, 255, 0.2)"
    )

# Add rectangles for each data point
for i in range(len(df)):
    fig.add_shape(
        type="rect",
        x0=df["best_x"][i] - 4*df["sigma_x"][i], x1=df["best_x"][i] + 4*df["sigma_x"][i],
        y0=df["best_y"][i] - 4*df["sigma_y"][i], y1=df["best_y"][i] + 4*df["sigma_y"][i],
        line=dict(color="green"),
        fillcolor="rgba(0, 255, 0, 0.2)"
    )
    
aper_trace = plot_2d_aperture(aper.APER_1.values[0]*1000, aper.APER_2.values[0]*1000, aper.APER_3.values[0]*1000, aper.APER_4.values[0]*1000, 'Aperture')
fig.add_trace(aper_trace)

fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='blue'), name='Input data', showlegend=True))
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='green'), name='Best fit data', showlegend=True))

# Update layout
fig.update_layout(
    xaxis_title="x [mm]",
    yaxis_title="y [mm]",
    showlegend=True,
    width=950,  # Set the width of the figure
    height=950  # Set the height of the figure
)

# Show plot
fig.show()